# IMPORT

In [1]:
import numpy as np
import pandas as pd

In [2]:
# การแก้ Fuzzy ว่าเอากี่ Percent ให้แก้จรง cutoff = x
import difflib

def fuzzy_merge(df1, df2, left_on, right_on, how='inner', cutoff=0.90):
    df_other= df2.copy()
    df_other[left_on] = [get_closest_match(x, df1[left_on], cutoff)
                         for x in df_other[right_on]]
    return df1.merge(df_other, on=left_on, how=how)

def get_closest_match(x, other, cutoff):
    matches = difflib.get_close_matches(x, other, cutoff=cutoff)
    return matches[0] if matches else None

In [3]:
def isEnglish(s):
  try:
    s.encode(encoding='utf-8').decode('ascii')
  except UnicodeDecodeError:
    return False
  else:
    return True

In [4]:
def clean_string(x):
  if pd.isna(x) == False:
    x = x.lower()
    x = x.strip()
    remove_list = [' ','ด็อกเตอร์','ดร.','ว่าที่','พล.อ.','พล.ท.','พล.ต.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต','ส.อ.',
                           'ส.ท.','ส.ต.',
                          'พลฯ','นนร.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','น.อ.','ร.น.','น.ท.','น.ต.','ร.อ.','ร.ท.','ร.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.',
                          'จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','นนอ.','นจอ.',
                          'พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.'
                           ,'พลเอก','พลโท','พลตรี','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','พลทหาร',
                        'พลเรือเอก','พลเรือโท','พลเรือตรี','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี',
                           'พลอากาศเอก' , 'พลอากาศโท','พลอากาศตรี','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท' ,'เรืออากาศตรี',
                           'พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี',
                           'mr.','mrs.','ms.','limited','inc.','co., ltd.,','ltd','(ไทย)','(ประเทศไทย)',
                        '','(',')','-','.',',','บริษัท','บจก','จำกัด','มหาชน','ห้างหุ้นส่วน','จำกัด','publiccompanylim','company','–','/' ,
                           'นาย','นางสาว' ,'นาง' , 'x0000' , '_' , 'และหรือย่อย']
    for i in range(len(remove_list)):
      x = x.replace(remove_list[i],'')
  return x

In [5]:
input_path = r'C:\Users\tharaphonn\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\Data\ปี 2566 Q4/'
output_path = r'C:\Users\tharaphonn\OneDrive - EXIM BANK OF THAILAND\Shared Drive บส ตส\3. รวมศูนย์\RESULT/'

# INPUT

In [6]:
#@title ข้อมูล Outstanding
#จาก Share Point
outstanding = '12.วงเงินและยอดคงค้าง_ธ.ค.2566.xlsx' #@param {type:'string'}
df_out = pd.read_excel(input_path + outstanding , dtype = str)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tharaphonn\\OneDrive - EXIM BANK OF THAILAND\\Shared Drive บส ตส\\Data\\ปี 2566 Q4/12.วงเงินและยอดคงค้าง_ธ.ค.2566.xlsx'

In [ ]:
#@title ชื่อไฟล์ รหัสลูกค้า เลขนิติบุคคล
#จาก Jupyter โดยการ Merge ข้อมูล วงเงินยอดคงค้าง กับ ไฟล์ข้อมูลลูกค้าในระบบ AS400
# Link Share Point : http://sp.exim.go.th/dc/report/Report14/Forms/AllItems.aspx
id_cuscode = 'รหัส_เลขนิติ_Q4_66.xlsx' #@param {type:'string'}
df_pm = pd.read_excel(input_path + id_cuscode , dtype = str)

In [ ]:
#@title ข้อมูล ECS File บัญชี
#ได้รับจากพี่หนุ่ม ภาคิน ทุกเดือน ฝ่าย พส.
ecs1 = 'ลูกค้าวงเงินหลักประกัน_บช_Dec23.xlsx' #@param {type:'string'}
ecs2 = 'เจ้าของหลักประกัน_audit_Dec23.xlsx' #@param {type:'string'}
df_ecs1 = pd.read_excel(input_path + ecs1 , dtype = str)
df_ecs2 = pd.read_excel(input_path + ecs2 , dtype = str)

In [ ]:
#@title ข้อมูลรวมศูนย์
# มีการให้ทุกต้นเดือนเป็น ออโต้จากฝ่าย พส.
centralize = 'รวมศูนย์_20240102.xlsx' #@param {type:'string'}
df_cen = pd.read_excel(input_path + centralize , dtype = str)

In [ ]:
#@title ใส่ขอบเขตการตรวจ
start_date = '2023-01-01' #@param {type:"date"}
end_date = '2023-12-31' #@param {type:"date"}
start_date = pd.to_datetime( start_date, format = '%Y-%m-%d')
end_date = pd.to_datetime(end_date  , format = '%Y-%m-%d')

In [ ]:
#@title ชื่อไฟล์ Output
file_name = 'รวมศูนย์_ผู้ค้ำประกัน_q4_66_jupyter.xlsx' #@param {type:'string'}

In [ ]:
#df_b = pd.read_excel(input_path + 'Carp_ทบทวนแล้ว_110865.xlsx' , dtype = str)

# CODE

## Chose Customer and Clean Outstanding File

In [ ]:
df_out1 = df_out[['รหัสลูกค้า' , 'ผู้ดูแลลูกค้า_(แยกส่วน)' , 'ผู้ดูแลลูกค้า_(ตามฝ่าย)' , 'วันที่อนุมัติ',
                           'วันที่ทบทวนล่าสุด']].drop_duplicates()

In [ ]:
def clean_date_out(a):
  if pd.isna(a) == False:
    if a[6:8] == '26':
      a = a.replace('265' , '256')
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '25':
      a = a[:6] + str(int(a[6:]) - 543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '20' or a[6:8] == '19':
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:7] == '3':
      a = a[:6] + str(int(a[6:]) -2*543)
      a = pd.to_datetime( a , format = '%d/%m/%Y')
    elif a[6:8] == '00':
      a = np.nan
  return a

In [ ]:
date_col = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด']
for i in range(len(date_col)):
  df_out1[date_col[i]] = df_out1[date_col[i]].apply(lambda x : clean_date_out(x))
 # df_out1[date_col[i]] = pd.to_datetime( df_out1[date_col[i]] , format = '%d/%m/%Y')

In [ ]:
df_out1 = df_out1.dropna(subset = ['ผู้ดูแลลูกค้า_(แยกส่วน)'])

In [ ]:
def check_date_interval(a , b , start_date , end_date):
  c = 0
  if pd.isna(a) == False:
    if (a >= start_date) and (a <= end_date):
      c +=1
  if pd.isna(b) == False:
    if (b >= start_date) and (b <= end_date):
      c +=1
  return c

In [ ]:
df_out1['is_in'] = df_out1.apply(lambda x : check_date_interval(x['วันที่อนุมัติ'] , x['วันที่ทบทวนล่าสุด'] , start_date , end_date) , axis = 1)

In [ ]:
df_out1.head()

In [ ]:
# Clean ข้อมูลให้เหลือ 1 บรรทัดก่อนของข้อมูล outstanding
cus_list = df_out1['รหัสลูกค้า'].unique().tolist()
df_out2 = pd.DataFrame()
for i in range(len(cus_list)):
  df_t = df_out1[df_out1['รหัสลูกค้า'] == cus_list[i]]

  date1_min = min(list(df_t['วันที่อนุมัติ']))
  date1_max = max(list(df_t['วันที่อนุมัติ']))

  #date2_min = min(list(df_t['วันที่ทบทวนล่าสุด']))
  date2_max = min(list(df_t['วันที่ทบทวนล่าสุด']))

  df_t1 = df_t.head(1)
  df_t1 = df_t1.drop(columns = ['วันที่อนุมัติ' , 'วันที่ทบทวนล่าสุด'])
  df_t1['วันที่อนุมัติครั้งแรก'] = date1_min
  df_t1['วันที่อนุมัติล่าสุด'] = date1_max
  #df_t1['วันที่ทบทวนครั้งแรก'] = date2_min
  df_t1['วันที่ทบทวนล่าสุด'] = date2_max

  df_out2 = pd.concat([df_out2 , df_t1])

In [ ]:
df_cusin = df_out1[df_out1['is_in'] != 0][['รหัสลูกค้า']].drop_duplicates()

In [ ]:
df_out3 = pd.merge(df_out2 , df_cusin , on = 'รหัสลูกค้า')

In [ ]:
print(len(df_out1) , len(df_out2) , len(df_out3))

In [ ]:
df_out4 = pd.merge(df_out3 , df_pm , on = 'รหัสลูกค้า' , how  ='left')

In [ ]:
#df_out4 = df_out4.drop(columns = ['is_in'])
df_out4.head()

## Clean ข้อมูล ECS

In [ ]:
df_ecs1.head()

In [ ]:
df_ecs2.head()

In [ ]:
df_e1 = df_ecs1[['CUST_CODE' , 'CR_LIMIT_NO' , 'COLL_TYPENAME' , 'COLL_DESCRIPTION' , 'COLLATERAL_NO']].drop_duplicates() #จากไฟล์ ECS บช
df_e2 = df_ecs2[['COLLATERAL_NO' , 'STATUS' , 'FIRSTNAME'  , 'LASTNAME' , 'REGISTRATION_NO']].drop_duplicates() # จากไฟล์ ECS ตส

In [ ]:
df_e3 = pd.merge(df_e1 , df_e2 , on = 'COLLATERAL_NO' , how = 'left')

In [ ]:
df_e3.head()

In [ ]:
def clean_ecs_name(fname , lname):
  name1 = np.nan
  name2 = np.nan
  if pd.isna(fname) == False:
    if fname == '-':
      fname = np.nan

  if pd.isna(lname) == False:
    if lname == '-':
      lname = np.nan

  if (pd.isna(fname) == False) and (pd.isna(lname) == False):
    if isEnglish(fname) == isEnglish(lname):
      name1 = fname + lname
      name2 = np.nan
    else:
      name1 = fname
      name2 = lname
  elif (pd.isna(fname) == False) and (pd.isna(lname) == True):
    name1 = fname
    name2 = np.nan
  elif (pd.isna(fname) == True) and (pd.isna(lname) == False):
    name1 = lname
    name2 = np.nan
  elif (pd.isna(fname) == False) and (pd.isna(lname) == False):
    name1 = np.nan
    name2 = np.nan
  return name1 , name2

In [ ]:
df_e3['NAME_1'] , df_e3['NAME_2'] = zip(* df_e3.apply(lambda x : clean_ecs_name( x['FIRSTNAME'] , x['LASTNAME']) , axis = 1 ))

In [ ]:
df_e3[df_e3['NAME_2'].notna()]

In [ ]:
name_col = ['NAME_1' , 'NAME_2']
for i in range(len(name_col)):
  df_e3[name_col[i]] = df_e3[name_col[i]].apply(lambda x : clean_string(x))

In [ ]:
df_e4 = pd.merge(df_e3 , df_out4 , left_on = 'CUST_CODE' , right_on = 'รหัสลูกค้า' )

In [ ]:
# เลือกขอบเขตแล้ว
df_e4.head()

In [ ]:
def is_debt(a):
  c = 0
  if pd.isna(a) == False:
    if ('หนี้' in a) or ('เงิน' in a):
      c = 1
  return c

In [ ]:
df_e4['is_debt'] = df_e4['ผู้ดูแลลูกค้า_(ตามฝ่าย)'].apply(lambda x : is_debt(x))

In [ ]:
df_e5 = df_e4[df_e4['is_debt'] == 0].drop(columns = ['is_debt'])

## Clean File รวมศูนย์

In [ ]:
df_cen['NAME_C1'] , df_cen['NAME_C2'] =  zip(* df_cen.apply(lambda x : clean_ecs_name( x['NAMETH'] , x['SURNAMETH']) , axis = 1 ))

In [ ]:
df_cen['NAME_C3'] , df_cen['NAME_C4'] =  zip(* df_cen.apply(lambda x : clean_ecs_name( x['NAMEEN'] , x['SURNAMEEN']) , axis = 1 ))

In [ ]:
name_col = ['NAME_C1' , 'NAME_C2' , 'NAME_C3' , 'NAME_C4']
for i in range(len(name_col)):
  df_cen[name_col[i]] = df_cen[name_col[i]].apply(lambda x : clean_string(x))

In [ ]:
df_cen.head()

## MERGE ครั้งละ รหัสลูกค้า

In [ ]:
# Loop ที่ใช้ตรวจ
cus_list = df_e5['CUST_CODE'].unique().tolist()
name_ecs = ['NAME_1' , 'NAME_2']
name_centralize = ['NAME_C1' , 'NAME_C2' ,'NAME_C3' , 'NAME_C4']
df_notin = pd.DataFrame()
for i in range(len(cus_list)):
  df_u = df_e5[df_e5['CUST_CODE'] == cus_list[i]]
  df_c = df_cen[df_cen['EXIM_CUSTID'] == cus_list[i]]
  df_m_out = df_u.copy()
  for m in range(len(name_ecs)):
    for n in range(len(name_centralize)):
      if (len(df_m_out.dropna(subset = [name_ecs[m]])) != 0) and len(df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates()):
        df_m = fuzzy_merge(df_m_out.dropna(subset = [name_ecs[m]]) ,
                                    df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates() ,
                                    left_on = name_ecs[m] , right_on = name_centralize[n] , how = 'left')
        df_m_out = df_m[df_m[name_centralize[n]].isna()].drop(columns = name_centralize[n])
  df_notin = pd.concat([df_notin , df_m_out])

In [ ]:
## สำหรับรหัส G จะยอมให้หาแต่ชื่อ

In [ ]:
def id_g(a):
  c = 0
  if pd.isna(a) == False:
    if a[0] == 'G':
      c = 1
  return c

In [ ]:
df_notin['is_g'] = df_notin['รหัสลูกค้า'].apply(lambda x : id_g(x))

In [ ]:
df_g = df_notin[df_notin['is_g'] == 1].drop(columns = ['is_g'])

df_notg = df_notin[df_notin['is_g'] == 0].drop(columns = ['is_g'])

In [ ]:
print(len(df_g) , len(df_notg))

In [ ]:
cus_list = df_g['CUST_CODE'].unique().tolist()
name_ecs = ['NAME_1' , 'NAME_2']
name_centralize = ['NAME_C1' , 'NAME_C2' ,'NAME_C3' , 'NAME_C4']
df_notin1 = pd.DataFrame()
for i in range(len(cus_list)):
  df_u = df_g[df_g['CUST_CODE'] == cus_list[i]]
  df_c = df_cen.copy()
  df_m_out = df_u.copy()
  for m in range(len(name_ecs)):
    for n in range(len(name_centralize)):
      if (len(df_m_out.dropna(subset = [name_ecs[m]])) != 0) and len(df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates()):
        df_m = fuzzy_merge(df_m_out.dropna(subset = [name_ecs[m]]) ,
                                    df_c[[name_centralize[n]]].dropna(subset = [name_centralize[n]]).drop_duplicates() ,
                                    left_on = name_ecs[m] , right_on = name_centralize[n] , how = 'left')
        df_m_out = df_m[df_m[name_centralize[n]].isna()].drop(columns = name_centralize[n])
  df_notin1 = pd.concat([df_notin1 , df_m_out])

In [ ]:
df_ans = pd.concat([df_notg , df_notin1])

In [ ]:
len(df_ans)

In [ ]:
df_ans

# EXPORT FILE

In [ ]:
df_ans.to_excel(output_path + file_name , index = False)